Notebook to save labels from the database in a zarr file.

In [1]:
import os
import sys
import zarr
import tensorstore as ts
from numcodecs import Blosc
import ome_zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image
import dask.array as da
from dask import delayed
import napari
import numpy as np
from tqdm.notebook import tqdm 

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.db.db_model import CellDB, TrackDB

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

from napari.settings import get_settings
settings = get_settings()
settings.experimental.async_
#settings.experimental.async_=True

False

In [3]:
# get images
ch0_path = r'D:\kasia\tracking\E6_exp\E6_C0.zarr'

im = da.from_zarr(ch0_path,1)
im_shape = im.shape
im_shape

(241, 8396, 8401)

In [4]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

In [5]:
def build_frame(session,im_shape,ind):

    frame = np.zeros([im_shape[1],im_shape[2]]).astype('uint32')

    query = session.query(CellDB).filter(CellDB.t == ind).all()

    for cell in query:

        box = frame[cell.bbox_0:cell.bbox_2,cell.bbox_1:cell.bbox_3]

        frame[cell.bbox_0:cell.bbox_2,cell.bbox_1:cell.bbox_3] = box + (cell.mask.astype('uint32') * cell.track_id)

    return frame

chunks = [1,2048,2048]

lazy_arrays = [delayed(build_frame)(session,im_shape,i) for i in range(im_shape[0])]
dask_arrays = [da.from_delayed(delayed_reader, shape=[im_shape[1],im_shape[2]], dtype='uint32') for delayed_reader in lazy_arrays]
stack = da.stack(dask_arrays, axis=0)
rechunked_stack = stack.rechunk(chunks)

In [6]:
rechunked_stack

dask.array<rechunk-merge, shape=(241, 8396, 8401), dtype=uint32, chunksize=(1, 2048, 2048), chunktype=numpy.ndarray>

In [ ]:
compressor = Blosc(cname='lz4', clevel=9, shuffle=Blosc.BITSHUFFLE)
zarr_store = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\labels_1024_from_db.zarr'

rechunked_stack.to_zarr(zarr_store, compressor=compressor)# 

In [ ]:
# alternatively
# compressor = Blosc(cname='lz4', clevel=9, shuffle=Blosc.BITSHUFFLE)

# # save zarr file

# size_t = 1
# size_xy = 1024

# zarr_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\labels_1024_from_db_ome.zarr'

# # write the image data
# store = parse_url(zarr_path, mode="w").store
# root = zarr.group(store=store)

# # it will fail if the store already contains arrays
# write_image(image=rechunked_stack, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))

### View saved zarr

In [ ]:
# get images
ch0_path = r'D:\kasia\tracking\E6_exp\E6_C0.zarr'
ch1_path = r'D:\kasia\tracking\E6_exp\E6_C1.zarr'

ch0_list = []
for level in range(1,5):
    ch0_list.append(da.from_zarr(ch0_path,level))

ch1_list = []
for level in range(1,5):
    ch1_list.append(da.from_zarr(ch1_path,level))

In [ ]:
labels_zarr_path = zarr_store

# open using a tensorstore

# Create the specification for opening the Zarr dataset
spec = {
  'driver': 'zarr',
  'kvstore': {
    'driver': 'file', 
    'path': labels_zarr_path,
  },
}

# Open the dataset asynchronously
dataset_future = ts.open(spec)

# Wait for the dataset to be fully opened (synchronous operation)
dataset = dataset_future.result()

In [6]:
# display channels and labels
viewer = napari.Viewer()

ch1 = viewer.add_image(ch0_list, name='ch1', colormap = 'green',blending='additive',contrast_limits=[0, 2048])
ch2 = viewer.add_image(ch1_list, name='ch2', colormap = 'red',blending='additive',contrast_limits=[0, 2048])

labels_layer = viewer.add_labels(dataset,name='Labels')